In [1]:
from collections import deque
from typing import List, Tuple

from pydantic import BaseModel, Field
from spondee.search import nlp_pipeline, identify_statements, sentence_metadata

In [2]:
nlp = nlp_pipeline()

2024-07-07 01:28:45 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-07-07 01:28:45 INFO: Downloaded file to /Users/tylerbrown/stanza_resources/resources.json
2024-07-07 01:28:45 WARNING: Language en package default expects mwt, which has been added
2024-07-07 01:28:45 INFO: Loading these models for language: en (English):
| Processor    | Package             |
--------------------------------------
| tokenize     | combined            |
| mwt          | combined            |
| pos          | combined_charlm     |
| constituency | ptb3-revised_charlm |

2024-07-07 01:28:45 INFO: Using device: cpu
2024-07-07 01:28:45 INFO: Loading: tokenize
2024-07-07 01:28:46 INFO: Loading: mwt
2024-07-07 01:28:46 INFO: Loading: pos
2024-07-07 01:28:46 INFO: Loading: constituency
2024-07-07 01:28:46 INFO: Done loading processors!


In [3]:
txt = "".join(
        [
            "Gavin Sheets hit his first career grand slam, and the ",
            "Chicago White Sox won their second straight after a ",
            "franchise-record 14-game losing streak, beating the ",
            "Boston Red Sox 6-1 on Saturday.",
        ]
    )

In [ ]:
#search_text(txt, nlp)

In [11]:
from spondee.schemas import Sentence

In [8]:
doc = nlp(txt)

In [41]:
for i, sentence in enumerate(doc.sentences):

    tree = sentence.constituency
    statements = identify_statements(tree)
    _slice = lambda x : (0,0) if len(x) == 0 else (x[0].start_char, x[-1].end_char)

    paths = sentence_metadata(i, statements, sentence.to_dict())
    for smeta in paths:

        start_char, end_char = _slice(smeta.subject)
        print(f"subject: {txt[start_char:end_char]}")
        
        start_char, end_char = _slice(smeta.predicate)
        print(f"predicate: {txt[start_char:end_char]}")

        for predicate_np in smeta.predicate_noun_phrases:
            start_char, end_char = _slice(predicate_np)
            print(f"predicate np: {txt[start_char:end_char]}")

        for subject_np in smeta.subject_noun_phrases:
            start_char, end_char = _slice(subject_np)
            print(f"subject np: {txt[start_char:end_char]}")

        print()

      
        #    sidx=smeta.sidx,
        #    subject=[ txt[get_slice(inst)] for inst in smeta.subject ],
        #    predicate=
        #)

subject: Gavin Sheets
predicate: hit his first career grand slam
predicate np: first career grand slam
subject np: Gavin Sheets

subject: the Chicago White Sox
predicate: won their second straight after a franchise-record 14-game losing streak, beating the Boston Red Sox 6-1 on Saturday
predicate np: second straight
predicate np: franchise-record 14-game losing streak
predicate np: Boston Red Sox
predicate np: Saturday
subject np: Chicago White Sox



In [24]:
paths[0].subject

def get_slice(meta):
    if len(meta) == 0:
        return slice(0, 0)

    return slice(meta[0].start_char, meta[-1].end_char)

In [ ]:
tree = docs.sentences[0].constituency
statements = identify_statements(tree)

In [ ]:
_paths = sentence_metadata(statements, docs.sentences[0].to_dict())

In [ ]:
for grp in _paths[1][3]:
    for m in grp:
        LeafLabel.model_validate(m)

In [ ]:
len(_paths)

In [ ]:
_paths[1][2][

In [15]:
"foo"[0:0]

''

In [ ]:
for grp in _paths[1][3]:
    for m in grp:
        try:
            LeafLabel.model_validate(m)
        except Exception as exc:
            print(m, exc)